## Extract data from CSV

In [206]:
import pandas as pd
import numpy as np
from supabase import create_client
from dotenv import load_dotenv
import os

data = pd.read_csv("./data/consumer_complaints.csv")

data.head(10)

C:\Users\USER\AppData\Local\Temp\ipykernel_25628\1893787032.py:7: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/consumer_complaints.csv")


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067
5,08/30/2013,Credit card,NaN,Application processing delay,NaN,NaN,NaN,Capital One,NY,12206,NaN,NaN,Phone,09/03/2013,Closed with explanation,Yes,Yes,510098
6,08/30/2013,Credit card,NaN,Credit line increase/decrease,NaN,NaN,NaN,Wells Fargo & Company,AZ,85730,NaN,NaN,Postal mail,09/05/2013,Closed with explanation,Yes,No,511062
7,08/30/2013,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Bank of America,IL,60660,NaN,NaN,Referral,09/04/2013,Closed with explanation,Yes,No,511116
8,08/30/2013,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Bank of America,GA,30016,NaN,NaN,Referral,09/04/2013,Closed with explanation,Yes,No,511091
9,09/17/2013,Mortgage,Conventional adjustable mortgage (ARM),"Loan modification,collection,foreclosure",NaN,NaN,NaN,"SunTrust Banks, Inc.",CA,94551,NaN,NaN,Web,09/18/2013,Closed with explanation,Yes,Yes,530602


## Transform Data

### Data Cleaning

In [207]:
print(data.isnull().sum())

# 2. Isi missing value per tipe kolom (menyesuaikan dataset)
# Isi sub_product dengan "Not Specified" jika null
data['sub_product'] = data['sub_product'].fillna('Not Specified')

# Isi sub_issue dengan "General" jika null
data['sub_issue'] = data['sub_issue'].fillna('General')

# Isi consumer_complaint_narrative dengan "No Narrative" jika null
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].fillna('No Narrative')

# Zipcode: Ambil 5 digit pertama, jika invalid atau null isi dengan "00000"
data['zipcode'] = data['zipcode'].apply(lambda x: str(x)[:5] if pd.notnull(x) else '00000')

# Isi company_public_response dengan "No Response" jika null
data['company_public_response'] = data['company_public_response'].fillna('No Response')

# Isi tags dengan "No Tag" jika null
data['tags'] = data['tags'].fillna('No Tag')

# Isi consumer_consent_provided dengan "Not Provided" jika null
data['consumer_consent_provided'] = data['consumer_consent_provided'].fillna('Not Provided')

# Isi complaint_id dengan -1 jika null (ID tidak boleh kosong)
if data['complaint_id'].isnull().any():
    data['complaint_id'] = data['complaint_id'].fillna(-1)

# 3. Cek lagi missing values setelah diisi
print("\nSetelah diisi:\n", data.isnull().sum())

date_received                        0
product                              0
sub_product                     158322
issue                                0
sub_issue                       343335
consumer_complaint_narrative    489151
company_public_response         470833
company                              0
state                             4887
zipcode                           4505
tags                            477998
consumer_consent_provided       432499
submitted_via                        0
date_sent_to_company                 0
company_response_to_consumer         0
timely_response                      0
consumer_disputed?                   0
complaint_id                         0
dtype: int64

Setelah diisi:
 date_received                      0
product                            0
sub_product                        0
issue                              0
sub_issue                          0
consumer_complaint_narrative       0
company_public_response            0
company  

### Data transformation

In [208]:
def generate_bigint():
    """Generates a valid BIGINT value"""
    return np.random.randint(1000000, 9999999)  

In [209]:


def transform_data(df):
    """Transform data into star schema dimensions and fact table"""
    
    # Generate BIGINT for each dimension key
    df['complaint_detail_key'] = df['complaint_id'].apply(lambda x: generate_bigint()) if 'complaint_id' in df.columns else [generate_bigint() for _ in range(len(df))]
    df['product_key'] = [generate_bigint() for _ in range(len(df))] if 'product' not in df.columns else df['product'].apply(lambda x: generate_bigint())
    df['company_key'] = [generate_bigint() for _ in range(len(df))] if 'company' not in df.columns else df['company'].apply(lambda x: generate_bigint())
    df['location_key'] = [generate_bigint() for _ in range(len(df))]
    df['issue_key'] = [generate_bigint() for _ in range(len(df))]
    df['response_key'] = [generate_bigint() for _ in range(len(df))]
    
    # Generate complaint_count for fact table
    df['complaint_count'] = 1
    
    # Handle missing or misnamed columns by using get() to avoid KeyError
    df['consumer_complaint_narrative'] = df.get('consumer_complaint_narrative', None)
    df['company_public_response'] = df.get('company_public_response', None)
    df['tags'] = df.get('tags', None)
    df['consumer_disputed'] = df.get('consumer_disputed', None)

    # Transform dim_complaint_detail
    dim_complaint_detail = df[['complaint_id', 'consumer_complaint_narrative', 'company_public_response', 'tags']].drop_duplicates()
    # Rename 'complaint_id' to 'complaint_detail_key' to match the schema
    dim_complaint_detail.rename(columns={'complaint_id': 'complaint_detail_key'}, inplace=True)
    
    # Transform dim_product
    dim_product = df[['product_key', 'product', 'sub_product']].drop_duplicates()
    
    # Transform dim_company
    dim_company = df[['company_key', 'company']].drop_duplicates()
    
    # Transform dim_location
    dim_location = df[['location_key', 'state', 'zipcode']].drop_duplicates()
    
    # Transform dim_issue
    dim_issue = df[['issue_key', 'issue', 'sub_issue']].drop_duplicates()

    # Transform dim_response
    dim_response = df[['response_key', 'submitted_via', 'company_response_to_consumer', 
                       'consumer_consent_provided', 'timely_response', 'consumer_disputed']].drop_duplicates()

    # Transform dim_date
    dim_date = pd.DataFrame()
    dim_date['date_key'] = pd.to_datetime(df['date_received']).dt.date  # Use 'date_received' here
    dim_date['full_date'] = pd.to_datetime(df['date_received'])  # Use 'date_received' here
    dim_date['year'] = dim_date['full_date'].dt.year
    dim_date['quarter'] = dim_date['full_date'].dt.quarter
    dim_date['month'] = dim_date['full_date'].dt.month
    dim_date['day'] = dim_date['full_date'].dt.day
    dim_date['day_of_week'] = dim_date['full_date'].dt.dayofweek
    dim_date['month_name'] = dim_date['full_date'].dt.month_name()
    dim_date['day_name'] = dim_date['full_date'].dt.day_name()
    dim_date['created_at'] = pd.to_datetime(df['created_at']) if 'created_at' in df.columns else None
    dim_date = dim_date.drop_duplicates()

    # Transform fact_complaints
    fact_complaints = df[['complaint_detail_key', 'date_received', 'date_sent_to_company', 
                           'product_key', 'company_key', 'location_key', 
                           'issue_key', 'response_key', 
                           'complaint_count']].drop_duplicates()
    
    # Since we are using BIGINT, there is no need to convert them to UUID
    # No UUID conversion needed, we can just merge using BIGINT
    fact_complaints = fact_complaints.merge(dim_complaint_detail[['complaint_detail_key']], 
                                            left_on='complaint_detail_key', right_on='complaint_detail_key', how='left')

    # Drop any unnecessary columns and ensure we have the correct 'complaint_detail_key'
    fact_complaints = fact_complaints.drop(columns=['complaint_detail_key'])
    fact_complaints = fact_complaints.rename(columns={'complaint_detail_key': 'complaint_detail_key'})

    # Ensure we are referencing the correct columns for date
    fact_complaints['date_received_key'] = pd.to_datetime(fact_complaints['date_received']).dt.date
    fact_complaints['date_sent_key'] = pd.to_datetime(fact_complaints['date_sent_to_company']).dt.date

    return dim_complaint_detail, dim_product, dim_company, dim_location, dim_issue, dim_response, dim_date, fact_complaints


## Load Data To Supabase Postgres

In [210]:
import os
from dotenv import load_dotenv
from supabase import create_client

def load_to_supabase(dim_complaint_detail, dim_product, dim_company, dim_location, dim_issue, dim_response, dim_date, fact_complaints):
    """Fungsi untuk memuat data ke Supabase"""
    print("Loading data to Supabase...")

    # Load environment variables
    load_dotenv()
    url = os.getenv("SUPABASE_URL")
    key = os.getenv("SUPABASE_KEY")

    # Initialize Supabase client
    supabase = create_client(url, key)

    def load_table(df, table_name):
        """Fungsi untuk memasukkan data ke Supabase dalam batch"""
        if df.empty:
            print(f"Warning: {table_name} is empty. Skipping.")
            return 0
        
        print(f"Loading {len(df)} records to {table_name}...")

        # Convert DataFrame to list of dictionaries (records)
        records = df.to_dict(orient='records')

        # Insert records in batches to avoid payload limits
        batch_size = 1000  # Adjust based on your data size
        total_loaded = 0

        for i in range(0, len(records), batch_size):
            batch = records[i:i+batch_size]
            try:
                # Perform upsert (insert or update) operation
                response = supabase.table(table_name).upsert(batch).execute()
                total_loaded += len(batch)
                print(f"  Loaded batch {i//batch_size + 1}, Progress: {total_loaded}/{len(records)}")
            except Exception as e:
                print(f"  Error loading batch to {table_name}: {e}")
                continue

        print(f"Completed loading {total_loaded}/{len(records)} records to {table_name}")
        return total_loaded

    # Load data into each table
    results = {
        "dim_complaint_detail": load_table(dim_complaint_detail, "dim_complaint_detail"),
        "dim_product": load_table(dim_product, "dim_product"),
        "dim_company": load_table(dim_company, "dim_company"),
        "dim_location": load_table(dim_location, "dim_location"),
        "dim_issue": load_table(dim_issue, "dim_issue"),
        "dim_response": load_table(dim_response, "dim_response"),
        "dim_date": load_table(dim_date, "dim_date"),
        "fact_complaints": load_table(fact_complaints, "fact_complaints")
    }

    # Print summary of data loading results
    print("\nData loading summary:")
    for table, count in results.items():
        print(f"- {table}: {count} records")

    return results


In [211]:
dim_complaint_detail, dim_product, dim_company, dim_location, dim_issue, dim_response, dim_date, fact_complaints = transform_data(data)
load_to_supabase(dim_complaint_detail, dim_product, dim_company, dim_location, dim_issue, dim_response, dim_date, fact_complaints)
print("Data transformation and loading completed successfully.")

Loading data to Supabase...
Loading 555957 records to dim_complaint_detail...
  Loaded batch 1, Progress: 1000/555957
  Loaded batch 2, Progress: 2000/555957
  Loaded batch 3, Progress: 3000/555957
  Loaded batch 4, Progress: 4000/555957
  Loaded batch 5, Progress: 5000/555957
  Loaded batch 6, Progress: 6000/555957
  Loaded batch 7, Progress: 7000/555957
  Loaded batch 8, Progress: 8000/555957
  Loaded batch 9, Progress: 9000/555957
  Loaded batch 10, Progress: 10000/555957
  Loaded batch 11, Progress: 11000/555957
  Loaded batch 12, Progress: 12000/555957
  Loaded batch 13, Progress: 13000/555957
  Loaded batch 14, Progress: 14000/555957
  Loaded batch 15, Progress: 15000/555957
  Loaded batch 16, Progress: 16000/555957
  Loaded batch 17, Progress: 17000/555957
  Loaded batch 18, Progress: 18000/555957
  Loaded batch 19, Progress: 19000/555957
  Loaded batch 20, Progress: 20000/555957
  Loaded batch 21, Progress: 21000/555957
  Loaded batch 22, Progress: 22000/555957
  Loaded batch 2

KeyboardInterrupt: 